1. # Credit card applications

Commercial banks receive a lot of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this notebook, we will build an automatic credit card approval predictor using machine learning techniques, just like the real banks do.

Credit card being held in hand

We'll use the [Credit Card Approval dataset](http://archive.ics.uci.edu/ml/datasets/credit+approval) from the UCI Machine Learning Repository. The structure of this notebook is as follows:

First, we will start off by loading and viewing the dataset.
We will see that the dataset has a mixture of both numerical and non-numerical features, that it contains values from different ranges, plus that it contains a number of missing entries.
We will have to preprocess the dataset to ensure the machine learning model we choose can make good predictions.
After our data is in good shape, we will do some exploratory data analysis to build our intuitions.
Finally, we will build a machine learning model that can predict if an individual's application for a credit card will be accepted.
First, loading and viewing the dataset. We find that since this data is confidential, the contributor of the dataset has anonymized the feature names.

Task 1: Instructions
Load and look at the dataset.

* Import the pandas library under the alias pd.
* Load the dataset, "datasets/cc_approvals.data", into a pandas DataFrame called cc_apps. Set the header argument to None.
* Print the first 5 rows of cc_apps using the head() method.

In [1]:
import pandas as pd 

cc_apps = pd.read_fwf("crx.data",header=None)

cc_apps.head(5)

,0
0,"b,30.83,0,u,g,w,v,1.25,t,t,01,f,g,00202,0,+"
1,"a,58.67,4.46,u,g,q,h,3.04,t,t,06,f,g,00043,560,+"
2,"a,24.50,0.5,u,g,q,h,1.5,t,f,0,f,g,00280,824,+"
3,"b,27.83,1.54,u,g,w,v,3.75,t,t,05,t,g,00100,3,+"
4,"b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+"


In [36]:
cc_apps=pd.read_csv("crx.data",sep=',',header=None)
cc_apps.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


2. # Inspecting the applications

The output may appear a bit confusing at its first sight, but let's try to figure out the most important features of a credit card application. The features of this dataset have been anonymized to protect the privacy, but this [blog](http://rstudio-pubs-static.s3.amazonaws.com/73039_9946de135c0a49daa7a0a9eda4a67a72.html) gives us a pretty good overview of the probable features. The probable features in a typical credit card application are Gender, Age, Debt, Married, BankCustomer, EducationLevel, Ethnicity, YearsEmployed, PriorDefault, Employed, CreditScore, DriversLicense, Citizen, ZipCode, Income and finally the ApprovalStatus. This gives us a pretty good starting point, and we can map these features with respect to the columns in the output.

As we can see from our first glance at the data, the dataset has a mixture of numerical and non-numerical features. This can be fixed with some preprocessing, but before we do that, let's learn about the dataset a bit more to see if there are other dataset issues that need to be fixed.

Task 2: Instructions
Inspect the structure, numerical summary, and specific rows of the dataset.

* Extract the summary statistics of the data using the describe() method of cc_apps.
* Use the info() method of cc_apps to get more information about the DataFrame.
* Print the last 17 rows of cc_apps using the tail() method to display missing values.

In [37]:
cc_apps.describe()

,2,7,10,14
count,690.000000,690.000000,690.00000,690.000000
mean,4.758725,2.223406,2.40000,1017.385507
std,4.978163,3.346513,4.86294,5210.102598
min,0.000000,0.000000,0.00000,0.000000
25%,1.000000,0.165000,0.00000,0.000000
50%,2.750000,1.000000,0.00000,5.000000
75%,7.207500,2.625000,3.00000,395.500000
max,28.000000,28.500000,67.00000,100000.000000


In [38]:
cc_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    object 
 13  13      690 non-null    object 
 14  14      690 non-null    int64  
 15  15      690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


In [39]:
cc_apps.tail(17)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
673,?,29.50,2.000,y,p,e,h,2.000,f,f,0,f,g,00256,17,-
674,a,37.33,2.500,u,g,i,h,0.210,f,f,0,f,g,00260,246,-
675,a,41.58,1.040,u,g,aa,v,0.665,f,f,0,f,g,00240,237,-
676,a,30.58,10.665,u,g,q,h,0.085,f,t,12,t,g,00129,3,-
677,b,19.42,7.250,u,g,m,v,0.040,f,t,1,f,g,00100,1,-
678,a,17.92,10.210,u,g,ff,ff,0.000,f,f,0,f,g,00000,50,-
679,a,20.08,1.250,u,g,c,v,0.000,f,f,0,f,g,00000,0,-
680,b,19.50,0.290,u,g,k,v,0.290,f,f,0,f,g,00280,364,-
681,b,27.83,1.000,y,p,d,h,3.000,f,f,0,f,g,00176,537,-
682,b,17.08,3.290,u,g,i,v,0.335,f,f,0,t,g,00140,2,-


3. # Handling the missing values (part i)
   
We've uncovered some issues that will affect the performance of our machine learning model(s) if they go unchanged:

* Our dataset contains both numeric and non-numeric data (specifically data that are of float64, int64 and object types). Specifically, the features 2, 7, 10 and 14 contain numeric values (of types float64, float64, int64 and int64 respectively) and all the other features contain non-numeric values.
* The dataset also contains values from several ranges. Some features have a value range of 0 - 28, some have a range of 2 - 67, and some have a range of 1017 - 100000. Apart from these, we can get useful statistical information (like mean, max, and min) about the features that have numerical values.
* Finally, the dataset has missing values, which we'll take care of in this task. The missing values in the dataset are labeled with '?', which can be seen in the last cell's output.
Now, let's temporarily replace these missing value question marks with NaN.

Task 3: Instructions

Inspect the missing values in the dataset and replace the question marks with NaN.

* Import the numpy library under the alias np.
* Print the last 17 rows of the dataset.
* Replace the '?'s with NaNs using the replace() method.
* Print the last 17 rows of cc_apps using the tail() method to confirm that the replace() method performed as expected.

In [40]:
import numpy as np

print(cc_apps.tail(17))

cc_apps.replace('?',np.nan,inplace=True)

print(cc_apps.tail(17))

    0      1       2  3  4   5   6      7  8  9   10 11 12     13   14 15
673  ?  29.50   2.000  y  p   e   h  2.000  f  f   0  f  g  00256   17  -
674  a  37.33   2.500  u  g   i   h  0.210  f  f   0  f  g  00260  246  -
675  a  41.58   1.040  u  g  aa   v  0.665  f  f   0  f  g  00240  237  -
676  a  30.58  10.665  u  g   q   h  0.085  f  t  12  t  g  00129    3  -
677  b  19.42   7.250  u  g   m   v  0.040  f  t   1  f  g  00100    1  -
678  a  17.92  10.210  u  g  ff  ff  0.000  f  f   0  f  g  00000   50  -
679  a  20.08   1.250  u  g   c   v  0.000  f  f   0  f  g  00000    0  -
680  b  19.50   0.290  u  g   k   v  0.290  f  f   0  f  g  00280  364  -
681  b  27.83   1.000  y  p   d   h  3.000  f  f   0  f  g  00176  537  -
682  b  17.08   3.290  u  g   i   v  0.335  f  f   0  t  g  00140    2  -
683  b  36.42   0.750  y  p   d   v  0.585  f  f   0  f  g  00240    3  -
684  b  40.58   3.290  u  g   m   v  3.500  f  f   0  t  s  00400    0  -
685  b  21.08  10.085  y  p   e   h  1

In [41]:
cc_apps.isnull().sum()

0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13    13
14     0
15     0
dtype: int64

4. # Handling the missing values (part ii)
   
We replaced all the question marks with NaNs. This is going to help us in the next missing value treatment that we are going to perform.

An important question that gets raised here is why are we giving so much importance to missing values? Can't they be just ignored? Ignoring missing values can affect the performance of a machine learning model heavily. While ignoring the missing values our machine learning model may miss out on information about the dataset that may be useful for its training. Then, there are many models which cannot handle missing values implicitly such as LDA.

So, to avoid this problem, we are going to impute the missing values with a strategy called mean imputation.

Impute the NaN values with the mean imputation approach.

* For the numeric columns, impute the missing values (NaNs) with pandas method fillna().
* Verify if the fillna() method performed as expected by printing the total number of NaNs in each column.
Remember that you have already marked all the question marks as NaNs. pandas provides fillna() to help you impute missing values with different strategies, mean imputation being one of them. pandas also has a mean() method to calculate the mean of a DataFrame. As your dataset contains both numeric and non-numeric data, for this task you will only impute the missing values (NaNs) present in the columns having numeric data-types (columns 2, 7, 10 and 14).

In [42]:
cc_apps.fillna(cc_apps.mean(),inplace=True)
cc_apps.isnull().sum()

0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13    13
14     0
15     0
dtype: int64

# 5. Handling the missing values (part iii)
   
We have successfully taken care of the missing values present in the numeric columns. There are still some missing values to be imputed for columns 0, 1, 3, 4, 5, 6 and 13. All of these columns contain non-numeric data and this why the mean imputation strategy would not work here. This needs a different treatment.

We are going to impute these missing values with the most frequent values as present in the respective columns. This is [good practice](https://www.datacamp.com/community/tutorials/categorical-data) when it comes to imputing missing values for categorical data in general.

Task 5: Instructions

Impute the missing values in the non-numeric columns.

* Iterate over each column of cc_apps using a for loop.
* Check if the data-type of the column is of object type by using the dtypes keyword.
* Using the fillna() method, impute the column's missing values with the most frequent value of that column with the value_counts() method and index attribute and assign it to cc_apps.
* Finally, verify if there are any more missing values in the dataset that are left to be imputed by printing the total number of NaNs in each column.


The column names of a pandas DataFrame can be accessed using columns attribute. The dtypes attribute provides the data type. In this part, object is the data type that you should be concerned about. The value_counts() method returns the frequency distribution of each value in the column, and the index attribute can then be used to get the most frequent value.

In [43]:
cc_apps[0].dtypes=='object'
cc_apps[0].value_counts().idxmax()

'b'

In [44]:
for col in cc_apps.columns:
    # Check if the column is of object type
    if cc_apps[col].dtype=='object':
        # Impute with the most frequent value
        cc_apps = cc_apps.fillna(cc_apps[0].value_counts().idxmax())

cc_apps.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
dtype: int64

In [45]:
cc_apps.nunique()

0       2
1     350
2     215
3       4
4       4
5      15
6      10
7     132
8       2
9       2
10     23
11      2
12      3
13    171
14    240
15      2
dtype: int64

# 6. Preprocessing the data (part i)

The missing values are now successfully handled.

There is still some minor but essential data preprocessing needed before we proceed towards building our machine learning model. We are going to divide these remaining preprocessing steps into three main tasks:

1. Convert the non-numeric data into numeric.
2. Split the data into train and test sets.
3. Scale the feature values to a uniform range.
   
First, we will be converting all the non-numeric values into numeric ones. We do this because not only it results in a faster computation but also many machine learning models (like XGBoost) (and especially the ones developed using scikit-learn) require the data to be in a strictly numeric format. We will do this by using a technique called [label encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

**Task 6: Instructions**

Convert the non-numeric values to numeric.

* Import the LabelEncoder class from sklearn.preprocessing module.
* Instantiate LabelEncoder() into a variable le.
* Iterate over all the values of each column cc_apps and check their data types using a for loop.
* If the data type is found to be of object type, label encode it to transform into numeric (such as int64) type.
* The values of each column a pandas DataFrame can be accessed using .columns and .to_numpy(). The dtypes attribute provides the data type. In this part, object is the data type that you should be concerned about.

In [46]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()

In [47]:
cc_apps[0].unique()

array(['b', 'a'], dtype=object)

In [51]:
for col in cc_apps.columns:
    # Compare if the dtypes is object
    if cc_apps[col].dtypes=='object':
        # Use LabelEncoder to do the numeric transformation
        cc_apps[col]=le.fit_transform(cc_apps[col].to_numpy())

In [52]:
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1,156,0.000,2,1,13,8,1.25,1,1,1,0,0,68,0,0
1,0,328,4.460,2,1,11,4,3.04,1,1,6,0,0,11,560,0
2,0,89,0.500,2,1,11,4,1.50,1,0,0,0,0,96,824,0
3,1,125,1.540,2,1,13,8,3.75,1,1,5,1,0,31,3,0
4,1,43,5.625,2,1,13,8,1.71,1,0,0,0,2,37,0,0


In [53]:
cc_apps.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
count,690.000000,690.000000,690.000000,690.000000,690.000000,690.000000,690.000000,690.000000,690.000000,690.000000,690.00000,690.000000,690.000000,690.000000,690.000000,690.000000
mean,0.695652,150.528986,4.758725,2.215942,1.466667,6.607246,5.994203,2.223406,0.523188,0.427536,2.40000,0.457971,0.176812,59.392754,1017.385507,0.555072
std,0.460464,96.188946,4.978163,0.477087,0.860126,4.412110,2.594506,3.346513,0.499824,0.495080,4.86294,0.498592,0.557869,48.231670,5210.102598,0.497318
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,71.000000,1.000000,2.000000,1.000000,2.000000,4.000000,0.165000,0.000000,0.000000,0.00000,0.000000,0.000000,23.000000,0.000000,0.000000
50%,1.000000,133.500000,2.750000,2.000000,1.000000,6.000000,8.000000,1.000000,1.000000,0.000000,0.00000,0.000000,0.000000,52.000000,5.000000,1.000000
75%,1.000000,226.000000,7.207500,2.000000,1.000000,11.000000,8.000000,2.625000,1.000000,1.000000,3.00000,1.000000,0.000000,96.000000,395.500000,1.000000
max,1.000000,349.000000,28.000000,3.000000,3.000000,14.000000,9.000000,28.500000,1.000000,1.000000,67.00000,1.000000,2.000000,170.000000,100000.000000,1.000000


In [56]:
cc_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    int32  
 1   1       690 non-null    int32  
 2   2       690 non-null    float64
 3   3       690 non-null    int32  
 4   4       690 non-null    int32  
 5   5       690 non-null    int32  
 6   6       690 non-null    int32  
 7   7       690 non-null    float64
 8   8       690 non-null    int32  
 9   9       690 non-null    int32  
 10  10      690 non-null    int64  
 11  11      690 non-null    int32  
 12  12      690 non-null    int32  
 13  13      690 non-null    int32  
 14  14      690 non-null    int64  
 15  15      690 non-null    int32  
dtypes: float64(2), int32(12), int64(2)
memory usage: 54.0 KB


# 7. Splitting the dataset into train and test sets

We have successfully converted all the non-numeric values to numeric ones.

Now, we will split our data into train set and test set to prepare our data for two different phases of machine learning modeling: training and testing. Ideally, no information from the test data should be used to scale the training data or should be used to direct the training process of a machine learning model. Hence, we first split the data and then apply the scaling.

Also, features like DriversLicense and ZipCode are not as important as the other features in the dataset for predicting credit card approvals. We should drop them to design our machine learning model with the best set of features. In Data Science literature, this is often referred to as feature selection.

**Task 7: Instructions**

Split the preprocessed dataset into train and test sets.

* Import train_test_split from the sklearn.model_selection module.
* Drop features 11 and 13 using the drop() method and convert the DataFrame to a NumPy array using .to_numpy().
* Segregate the features and labels into X and y (the column with index 13 is the label column).
* Using the train_test_split() method, split the data into train and test sets with a split ratio of 33% (test_size argument) and set the random_state argument to 42.
  
A NumPy array can be segregated using array slicing. Before slicing, take note of the total number of columns that should be present in the array after dropping features 11 and 13.

Setting random_state ensures the dataset is split with same sets of instances every time the code is run.

In [61]:
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1,156,0.000,2,1,13,8,1.25,1,1,1,0,0,68,0,0
1,0,328,4.460,2,1,11,4,3.04,1,1,6,0,0,11,560,0
2,0,89,0.500,2,1,11,4,1.50,1,0,0,0,0,96,824,0
3,1,125,1.540,2,1,13,8,3.75,1,1,5,1,0,31,3,0
4,1,43,5.625,2,1,13,8,1.71,1,0,0,0,2,37,0,0


In [62]:
from sklearn.model_selection import train_test_split


# Drop the features 11 and 13 and convert the DataFrame to a NumPy array
cc_apps = cc_apps.drop([11, 13], axis=1)
cc_apps = cc_apps.values

In [67]:
# Segregate features and labels into separate variables
X,y = cc_apps[:,0:13] , cc_apps[:,13:]

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                y,
                                test_size=0.33,
                                random_state=42)

# 8. Preprocessing the data (part ii)

The data is now split into two separate sets - train and test sets respectively. We are only left with one final preprocessing step of scaling before we can fit a machine learning model to the data.

Now, let's try to understand what these scaled values mean in the real world. Let's use CreditScore as an example. The credit score of a person is their creditworthiness based on their credit history. The higher this number, the more financially trustworthy a person is considered to be. So, a CreditScore of 1 is the highest since we're rescaling all the values to the range of 0-1.

**Task 8: Instructions**

Drop DriversLicense and ZipCode features and rescale the data.

* Import the MinMaxScaler class from the sklearn.preprocessing module.
* Instantiate MinMaxScaler class in a variable called scaler with the feature_range parameter set to (0,1).
* Fit the scaler to X_train and transform the data, assigning the result to rescaledX_train.
* Use the scaler to transform X_test, assigning the result to rescaledX_test.

When a dataset has varying ranges as in this credit card approvals dataset, one a small change in a particular feature may not have a significant effect on the other feature, which can cause a lot of problems when predictive modeling.

In [68]:
from sklearn.preprocessing import MinMaxScaler

# Instantiate MinMaxScaler and use it to rescale X_train and X_test
scaler=MinMaxScaler(feature_range=(0,1))

rescaledX_train=scaler.fit_transform(X_train)
rescaledX_test=scaler.fit_transform(X_test)

# 9. Fitting a logistic regression model to the train set

Essentially, predicting if a credit card application will be approved or not is a classification task. According to UCI, our dataset contains more instances that correspond to "Denied" status than instances corresponding to "Approved" status. Specifically, out of 690 instances, there are 383 (55.5%) applications that got denied and 307 (44.5%) applications that got approved.

This gives us a benchmark. A good machine learning model should be able to accurately predict the status of the applications with respect to these statistics.

Which model should we pick? A question to ask is: are the features that affect the credit card approval decision process correlated with each other? Although we can measure correlation, that is outside the scope of this notebook, so we'll rely on our intuition that they indeed are correlated for now. Because of this correlation, we'll take advantage of the fact that generalized linear models perform well in these cases. Let's start our machine learning modeling with a Logistic Regression model (a generalized linear model).

**Task 9: Instructions**

Fit a LogisticRegression classifier with rescaledX_train and y_train.

* Import LogisticRegression from the sklearn.linear_model module.
* Instantiate LogisticRegression into a variable named logreg with default values.
* Fit rescaledX_train and y_train to logreg using the fit() method.

If a quick refresher on logistic regression's working mechanism is needed, check out this tutorial.


In [69]:
from sklearn.linear_model import LogisticRegression

logreg=LogisticRegression()

logreg.fit(rescaledX_train,y_train)

C:\Users\aduzo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression()

# 10. Making predictions and evaluating performance

But how well does our model perform?

We will now evaluate our model on the test set with respect to [classification accuracy](https://developers.google.com/machine-learning/crash-course/classification/accuracy). But we will also take a look the model's [confusion matrix](https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/). In the case of predicting credit card applications, it is equally important to see if our machine learning model is able to predict the approval status of the applications as denied that originally got denied. If our model is not performing well in this aspect, then it might end up approving the application that should have been approved. The confusion matrix helps us to view our model's performance from these aspects.

**Task 10: Instructions**

Make predictions and evaluate performance.

* Import confusion_matrix() from sklearn.metrics module.
* Use predict() on rescaledX_test (which contains instances of the dataset that logreg has not seen until now) and store the predictions in a variable named y_pred.
* Print the accuracy score of logreg using the score(). Don't forget to pass rescaledX_test and y_test to the score() method.
* Call confusion_matrix() with y_test and y_pred to print the confusion matrix.



In [70]:
from sklearn.metrics import confusion_matrix

# Use logreg to predict instances from the test set and store it
y_pred = logreg.predict(rescaledX_test)

# Get the accuracy score of logreg model and print it
print("Accuracy of logistic regression classifier: ", logreg.score(rescaledX_test,y_test))

# Print the confusion matrix of the logreg model
# ... YOUR CODE FOR TASK 10 ...

confusion_matrix(y_test,y_pred)

Accuracy of logistic regression classifier:  0.8377192982456141


array([[92, 11],
       [26, 99]], dtype=int64)

# 11. Grid searching and making the model perform better

Our model was pretty good! It was able to yield an accuracy score of almost 84%.

For the confusion matrix, the first element of the of the first row of the confusion matrix denotes the true negatives meaning the number of negative instances (denied applications) predicted by the model correctly. And the last element of the second row of the confusion matrix denotes the true positives meaning the number of positive instances (approved applications) predicted by the model correctly.

Let's see if we can do better. We can perform a [grid search](https://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/) of the model parameters to improve the model's ability to predict credit card approvals.

[scikit-learn's implementation of logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) consists of different hyperparameters but we will grid search over the following two:

tol
max_iter

**Task 11: Instructions**

Define the grid of parameter values for which grid searching is to be performed.

* Import GridSearchCV from the sklearn.model_selection module.
* Define the grid of values for tol and max_iter parameters into tol and max_iter lists respectively.
* For tol, define the list with values 0.01, 0.001 and 0.0001. For max_iter, define the list with values 100, 150 and 200.
* Using the dict() method, create a dictionary where tol and max_iter are keys, and the lists of their values are the corresponding values. Name this dictionary as param_grid.
Grid search can be very exhaustive if the model is very complex and the dataset is extremely large. Luckily, that is not the case for this project.

In [73]:
from sklearn.model_selection import GridSearchCV

# Define the grid of values for tol and max_iter
tol = [0.01,0.001,0.0001]
max_iter = [100,150,200]

# Create a dictionary where tol and max_iter are keys and the lists of their values are corresponding values
param_grid = dict(tol=tol, max_iter=max_iter)
param_grid

{'tol': [0.01, 0.001, 0.0001], 'max_iter': [100, 150, 200]}

# 12. Finding the best performing model

We have defined the grid of hyperparameter values and converted them into a single dictionary format which GridSearchCV() expects as one of its parameters. Now, we will begin the grid search to see which values perform best.

We will instantiate GridSearchCV() with our earlier logreg model with all the data we have. Instead of passing train and test sets separately, we will supply X (scaled version) and y. We will also instruct GridSearchCV() to perform a cross-validation of five folds.

We'll end the notebook by storing the best-achieved score and the respective best parameters.

While building this credit card predictor, we tackled some of the most widely-known preprocessing steps such as scaling, label encoding, and missing value imputation. We finished with some machine learning to predict if a person's application for a credit card would get approved or not given some information about that person.

**Task 12: Instructions**

Find the best score and best parameters for the model using grid search.

* Instantiate GridSearchCV() with the attributes set as estimator = logreg, param_grid = param_grid and cv = 5 and store this instance in grid_model variable.
* Use scaler (which you created in Task-8) rescale X and assign it to rescaledX.
* Fit rescaledX and y to grid_model and store the results in grid_model_result.
* Call the best_score_ and best_params_ attributes on the grid_model_result variable, then print both.


Grid searching is a process of finding an optimal set of values for the parameters of a certain machine learning model. This is often known as hyperparameter optimization which is an active area of research. Note that, here we have used the word parameters and hyperparameters interchangeably, but they are not exactly the same.


In [74]:
# Instantiate GridSearchCV with the required parameters
grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)

# Use scaler to rescale X and assign it to rescaledX
rescaledX = scaler.fit_transform(X)

# Fit data to grid_model
grid_model_result = grid_model.fit(rescaledX, y)

# Summarize results
best_score, best_params = grid_model_result.best_score_, grid_model_result.best_params_
print("Best: %f using %s" % (best_score, best_params))

C:\Users\aduzo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\aduzo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\aduzo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\aduzo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using r

Best: 0.850725 using {'max_iter': 100, 'tol': 0.01}
